In [1]:
from glob import glob
import numpy as np
from astropy.table import QTable
import astropy.units as u
from astropy.io import fits
from astropy import stats
from scipy.ndimage import binary_dilation  
from astropy.modeling import models, fitting
import numpy as np
from astropy.modeling import models, fitting
from reproject import reproject_interp

def fit_2d_gaussian_and_get_sum(image):

    image = np.squeeze(image)
    image[np.isnan(image)] = 0

    # Get the center of the image
    shape_x, shape_y = image.shape
    center_x, center_y = np.array(image.shape) // 2
    
    # Create x, y indices grid for the sub-image
    x, y = np.array(np.mgrid[:shape_x, :shape_y], dtype=np.int32)
    
    # Initialize the Gaussian2D model
    g_init = models.Gaussian2D(amplitude=np.nanmax(image), x_mean=center_x, y_mean=center_y)
    
    # Fit the model to the sub-image
    fit_g = fitting.LevMarLSQFitter()
    g = fit_g(g_init, x, y, image)
    
    # Calculate the sum of the fitted Gaussian
    fitted_data = np.array(g(x, y), dtype=np.float32)
    gaussian_sum = np.sum(fitted_data)
    
    return (gaussian_sum*u.Jy, fitted_data)

def remove_nan_padding(data):
    
    # Find valid data indices along each axis
    valid_x = np.where(np.nansum(data, axis=0)!=0)[0]
    valid_y = np.where(np.nansum(data, axis=1)!=0)[0]

    # In the rare case there's still no valid data
    if len(valid_x) == 0 or len(valid_y) == 0:
        return data
    
    # Crop the data array
    cropped_data = data[valid_y[0]:valid_y[-1]+1, valid_x[0]:valid_x[-1]+1]
       
    return cropped_data

In [6]:
which = 'gaussians'

which_time = ''
# which_time = '_6totaltime'
# which_time = '_60totaltime'
# which_time = '_6totaltime_flagged'
# which_time = '_60totaltime_flagged'

dir_sim = f'./../data/{which}_input/'
dir_obs = f'./../data/{which}_observed{which_time}/'

files_sim = glob(f'{dir_sim}*.fits')
files_obs = glob(f'{dir_obs}*.pbcor.Jyperpix.fits')

files_sim.sort()
files_obs.sort()

files_sim, files_obs

(['./../data/gaussians_input/conf1_11.4mrs0.fits',
  './../data/gaussians_input/conf1_14.2mrs0.fits',
  './../data/gaussians_input/conf1_17.1mrs0.fits',
  './../data/gaussians_input/conf1_19.9mrs0.fits',
  './../data/gaussians_input/conf1_2.9mrs0.fits',
  './../data/gaussians_input/conf1_22.8mrs0.fits',
  './../data/gaussians_input/conf1_25.7mrs0.fits',
  './../data/gaussians_input/conf1_28.5mrs0.fits',
  './../data/gaussians_input/conf1_31.4mrs0.fits',
  './../data/gaussians_input/conf1_34.2mrs0.fits',
  './../data/gaussians_input/conf1_37.1mrs0.fits',
  './../data/gaussians_input/conf1_39.9mrs0.fits',
  './../data/gaussians_input/conf1_42.8mrs0.fits',
  './../data/gaussians_input/conf1_5.7mrs0.fits',
  './../data/gaussians_input/conf1_8.6mrs0.fits'],
 ['./../data/gaussians_observed/conf1_11.4mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits',
  './../data/gaussians_observed/conf1_14.2mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits',
  './../data/gaussians_observed/conf1_17.1mrs0.alma.cycle5.

In [14]:
confs = ['conf1']
for conf in confs: 

    n = 15
    max_sim = ['']*n
    max_obs = ['']*n
    sum_sim = ['']*n
    sum_obs = ['']*n
    sum_mask10_obs = ['']*n
    sum_mask10_sim = ['']*n
    sum_mask50_obs = ['']*n
    sum_mask50_sim = ['']*n
    sum_fit_sim = ['']*n
    sum_fit_obs = ['']*n
    rms_arr = ['']*n
    conf_arr = ['']*n
    wide_arr = ['']*n

    for i, file_sim in enumerate(files_sim):

        conf_file = file_sim.split('/')[-1].split('_')[0]
        wide_file = file_sim.split('/')[-1].split('_')[-1].replace('.fits', '')
        
        for file_obs in files_obs:
            if (conf_file in file_obs) & (wide_file in file_obs):

                if conf_file != conf: 
                    continue
                
                if float(conf.replace('conf', '')) not in [1]: 
                    continue

                conf_arr[i] = conf
                wide_arr[i] = wide_file

                print(file_sim.split('/')[-1], file_obs.split('/')[-1])

                data_sim = np.array(np.squeeze(fits.getdata(file_sim)), dtype=np.float32)
                data_obs = np.array(np.squeeze(fits.getdata(file_obs)), dtype=np.float32)

                data_obs = remove_nan_padding(data_obs)

                rms_sim = 0  
                rms_obs = stats.mad_std(data_obs, ignore_nan=True)
                rms_obs = stats.mad_std(data_obs[data_obs<rms_obs], ignore_nan=True)
                rms_arr[i] = rms_obs*u.Jy

                mask_high = data_obs == np.nanmax(data_obs)
                mask_low = data_obs > rms_obs*3
                mask = binary_dilation(mask_high, mask=mask_low, iterations=-1)

                rms_hdu = fits.PrimaryHDU(mask*np.int32(1), fits.getheader(file_obs))
                rms_hdu.writeto(file_obs.replace('.Jyperpix.fits', '.Jyperpix.mask.fits'), overwrite=True)

                max_sim[i] = np.nanmax(data_sim)*u.Jy
                max_obs[i] = np.nanmax(data_obs)*u.Jy

                sum_sim[i] = np.nansum(data_sim)*u.Jy
                sum_obs[i] = np.nansum(data_obs[mask])*u.Jy

                sum_fit_sim[i], _ = fit_2d_gaussian_and_get_sum(data_sim)
                sum_fit_obs[i], fitted_data = fit_2d_gaussian_and_get_sum(data_obs) 

                ### Get masked flux
                hdu_sim = fits.open(file_sim)[0]
                hdu_obs = fits.open(file_obs)[0]

                data_sim_r, _ = reproject_interp(hdu_sim, hdu_obs.header)

                mask_sim = hdu_sim.data/np.nanmax(hdu_sim.data) > 0.1
                mask_sim_r = data_sim_r/np.nanmax(data_sim_r) > 0.1
                
                sum_mask10_sim[i] = np.nansum(hdu_sim.data[mask_sim])*u.Jy
                sum_mask10_obs[i] = np.nansum(hdu_obs.data[mask_sim_r])*u.Jy

                mask_sim = hdu_sim.data/np.nanmax(hdu_sim.data) > 0.5
                mask_sim_r = data_sim_r/np.nanmax(data_sim_r) > 0.5
                
                sum_mask50_sim[i] = np.nansum(hdu_sim.data[mask_sim])*u.Jy
                sum_mask50_obs[i] = np.nansum(hdu_obs.data[mask_sim_r])*u.Jy

    ### Organise into astropy table 
    for i in range(len(sum_sim)):
        if sum_mask10_sim[i] == '':
            sum_fit_sim[i] = np.nan *u.Jy
            sum_fit_obs[i] = np.nan *u.Jy
            sum_mask10_sim[i] = np.nan *u.Jy
            sum_mask10_obs[i] = np.nan *u.Jy
            sum_mask50_sim[i] = np.nan *u.Jy
            sum_mask50_obs[i] = np.nan *u.Jy
            sum_sim[i] = np.nan *u.Jy
            sum_obs[i] = np.nan *u.Jy
            rms_arr[i] = np.nan *u.Jy
            max_obs[i] = np.nan *u.Jy
            max_sim[i] = np.nan *u.Jy

    table = QTable([conf_arr, 
                wide_arr, 
                sum_sim, 
                sum_obs, 
                rms_arr, 
                max_sim,
                max_obs,
                sum_fit_sim, 
                sum_fit_obs,
                sum_mask10_sim, 
                sum_mask10_obs,
                sum_mask50_sim, 
                sum_mask50_obs,
                ], 
            names=('conf', 
                'wide', 
                'sum_sim', 
                'sum_obs', 
                'rms_obs', 
                'max_sim',
                'max_obs',
                'sum_fit_sim', 
                'sum_fit_obs',
                'sum_mask10_sim', 
                'sum_mask10_obs',
                'sum_mask50_sim', 
                'sum_mask50_obs'))

    conf_ = table['conf'].copy()
    wide_ = table['wide'].copy()

    for i, (conf, wide) in enumerate(zip(conf_, wide_)):

        if conf == '':
            conf_[i] = np.nan
            wide_[i] = np.nan
        else:
            conf_[i] = float(conf.replace('conf', ''))
            wide_[i] = float(wide.replace('mrs0', ''))

    table['conf_'] = np.array(conf_, dtype=float)
    table['wide_'] = np.array(wide_, dtype=float)
    table.sort(['conf_', 'wide_'], reverse=False) # sort

    ids_nan = np.where(np.isnan(table['conf_']))[0]
    table.remove_rows(ids_nan)

    table.write(f'../data/tables/table_fit_{which}{which_time}_{conf}.fits', overwrite=True)
    table.write(f'../data/tables/table_fit_{which}{which_time}_{conf}.csv', overwrite=True)

conf1_11.4mrs0.fits conf1_11.4mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


conf1_14.2mrs0.fits conf1_14.2mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


conf1_17.1mrs0.fits conf1_17.1mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits


...

Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


conf1_5.7mrs0.fits conf1_5.7mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


conf1_8.6mrs0.fits conf1_8.6mrs0.alma.cycle5.1.image.pbcor.Jyperpix.fits


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
